# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Poroshkovo,UA,48.6683,22.7533,48.90,100,100,3.69,overcast clouds
1,1,Avarua,CK,-21.2078,-159.7750,77.05,69,40,17.27,scattered clouds
2,2,Mahibadhoo,MV,3.7833,72.9667,81.32,78,100,18.92,overcast clouds
3,3,New Norfolk,AU,-42.7826,147.0587,49.91,70,100,1.99,overcast clouds
4,4,Khatanga,RU,71.9667,102.5000,6.40,99,84,3.00,broken clouds
5,5,Saint-Philippe,RE,-21.3585,55.7679,69.39,79,53,10.00,broken clouds
6,6,Mataura,NZ,-46.1927,168.8643,38.48,89,100,8.08,overcast clouds
7,7,Hilo,US,19.7297,-155.0900,86.29,86,19,1.01,few clouds
8,8,Rikitea,PF,-23.1203,-134.9692,69.01,64,0,22.28,clear sky
9,9,Puerto Ayora,EC,-0.7393,-90.3518,69.55,88,76,5.99,broken clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))


# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df =city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Avarua,CK,-21.2078,-159.7750,77.05,69,40,17.27,scattered clouds
2,2,Mahibadhoo,MV,3.7833,72.9667,81.32,78,100,18.92,overcast clouds
7,7,Hilo,US,19.7297,-155.0900,86.29,86,19,1.01,few clouds
10,10,Vaini,TO,-21.2000,-175.2000,76.48,87,100,13.56,light rain
12,12,Kongolo,CD,-5.3833,27.0000,75.90,66,92,3.44,overcast clouds
23,23,Kalmunai,LK,7.4167,81.8167,79.39,79,100,4.14,overcast clouds
27,27,Kavieng,PG,-2.5744,150.7967,81.93,77,100,13.22,light rain
28,28,Bandarbeyla,SO,9.4942,50.8122,77.41,83,72,18.01,broken clouds
31,31,Half Moon Bay,US,37.4636,-122.4286,83.53,74,100,6.91,overcast clouds
41,41,Faanui,PF,-16.4833,-151.7500,77.59,75,40,26.31,light rain


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [5]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()
# Display sample data
clean_travel_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Avarua,CK,-21.2078,-159.7750,77.05,69,40,17.27,scattered clouds
2,2,Mahibadhoo,MV,3.7833,72.9667,81.32,78,100,18.92,overcast clouds
7,7,Hilo,US,19.7297,-155.0900,86.29,86,19,1.01,few clouds
10,10,Vaini,TO,-21.2000,-175.2000,76.48,87,100,13.56,light rain
12,12,Kongolo,CD,-5.3833,27.0000,75.90,66,92,3.44,overcast clouds
23,23,Kalmunai,LK,7.4167,81.8167,79.39,79,100,4.14,overcast clouds
27,27,Kavieng,PG,-2.5744,150.7967,81.93,77,100,13.22,light rain
28,28,Bandarbeyla,SO,9.4942,50.8122,77.41,83,72,18.01,broken clouds
31,31,Half Moon Bay,US,37.4636,-122.4286,83.53,74,100,6.91,overcast clouds
41,41,Faanui,PF,-16.4833,-151.7500,77.59,75,40,26.31,light rain


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [6]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp","Current Description","Lat", "Lng"]].copy()

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng
1,Avarua,CK,77.05,scattered clouds,-21.2078,-159.7750
2,Mahibadhoo,MV,81.32,overcast clouds,3.7833,72.9667
7,Hilo,US,86.29,few clouds,19.7297,-155.0900
10,Vaini,TO,76.48,light rain,-21.2000,-175.2000
12,Kongolo,CD,75.90,overcast clouds,-5.3833,27.0000
23,Kalmunai,LK,79.39,overcast clouds,7.4167,81.8167
27,Kavieng,PG,81.93,light rain,-2.5744,150.7967
28,Bandarbeyla,SO,77.41,broken clouds,9.4942,50.8122
31,Half Moon Bay,US,83.53,overcast clouds,37.4636,-122.4286
41,Faanui,PF,77.59,light rain,-16.4833,-151.7500


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [7]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Avarua,CK,77.05,scattered clouds,-21.2078,-159.7750,
2,Mahibadhoo,MV,81.32,overcast clouds,3.7833,72.9667,
7,Hilo,US,86.29,few clouds,19.7297,-155.0900,
10,Vaini,TO,76.48,light rain,-21.2000,-175.2000,
12,Kongolo,CD,75.90,overcast clouds,-5.3833,27.0000,
...,...,...,...,...,...,...,...
702,Takoradi,GH,76.08,clear sky,4.8845,-1.7554,
703,Lufkin,US,86.85,clear sky,31.3382,-94.7291,
709,Abu Dhabi,AE,86.97,clear sky,24.4667,54.3667,
710,Piacabucu,BR,79.03,clear sky,-10.4056,-36.4344,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [8]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [9]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# Display sample data
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Avarua,CK,77.05,scattered clouds,-21.2078,-159.7750,Paradise Inn
2,Mahibadhoo,MV,81.32,overcast clouds,3.7833,72.9667,Omadhoo Inn
7,Hilo,US,86.29,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
10,Vaini,TO,76.48,light rain,-21.2000,-175.2000,Keleti Beach Resort
12,Kongolo,CD,75.90,overcast clouds,-5.3833,27.0000,GUEST HOUSE OASIS
...,...,...,...,...,...,...,...
702,Takoradi,GH,76.08,clear sky,4.8845,-1.7554,Raybow International Hotel
703,Lufkin,US,86.85,clear sky,31.3382,-94.7291,Courtyard by Marriott Lufkin
709,Abu Dhabi,AE,86.97,clear sky,24.4667,54.3667,Beach Rotana
710,Piacabucu,BR,79.03,clear sky,-10.4056,-36.4344,Piaçabuçu News


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [14]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]

# Display sample data
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Avarua,CK,77.05,scattered clouds,-21.2078,-159.7750,Paradise Inn
2,Mahibadhoo,MV,81.32,overcast clouds,3.7833,72.9667,Omadhoo Inn
7,Hilo,US,86.29,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
10,Vaini,TO,76.48,light rain,-21.2000,-175.2000,Keleti Beach Resort
12,Kongolo,CD,75.90,overcast clouds,-5.3833,27.0000,GUEST HOUSE OASIS
...,...,...,...,...,...,...,...
702,Takoradi,GH,76.08,clear sky,4.8845,-1.7554,Raybow International Hotel
703,Lufkin,US,86.85,clear sky,31.3382,-94.7291,Courtyard by Marriott Lufkin
709,Abu Dhabi,AE,86.97,clear sky,24.4667,54.3667,Beach Rotana
710,Piacabucu,BR,79.03,clear sky,-10.4056,-36.4344,Piaçabuçu News


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [18]:
# In case of an empty DataFrame, load the sample data provided
sample_hotel_df = pd.read_csv("../WeatherPy_vacation-backup.csv")
sample_hotel_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,Aksu,CN,82.42,clear sky,41.12,80.26,Pudong Holiday Hotel
1,7,Touros,BR,80.60,broken clouds,-5.20,-35.46,INN NEW HORIZON
2,10,Morehead,US,75.20,clear sky,37.27,-87.18,CCI Express Inn
3,11,Port Elizabeth,ZA,84.20,clear sky,-33.92,25.57,39 On Nile Guest House
4,16,Northam,GB,82.40,few clouds,51.03,-4.22,Durrant House Hotel
5,18,Hyderabad,IN,87.01,haze,17.38,78.47,"Taj Krishna, Hyderabad"
6,20,Port Alfred,ZA,82.00,clear sky,-33.59,26.89,The Halyards Hotel
7,22,Atuona,PF,79.72,clear sky,-9.80,-139.03,Villa Enata
8,23,Kapaa,US,73.40,heavy intensity rain,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
9,25,Nikolskoye,RU,88.00,clear sky,59.70,30.79,Tourist House - Sablino


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [22]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"
# Export the city_data DataFrame into a CSV file
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [23]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [24]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Paradise Inn</dd>\n<dt>City</dt><dd>Avarua</dd>\n<dt>Country</dt><dd>CK</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 77.05 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Omadhoo Inn</dd>\n<dt>City</dt><dd>Mahibadhoo</dd>\n<dt>Country</dt><dd>MV</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 81.32 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hilo Hawaiian Hotel</dd>\n<dt>City</dt><dd>Hilo</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>few clouds and 86.29 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Keleti Beach Resort</dd>\n<dt>City</dt><dd>Vaini</dd>\n<dt>Country</dt><dd>TO</dd>\n<dt>Current Weather</dt><dd>light rain and 76.48 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>GUEST HOUSE OASIS</dd>\n<dt>City</dt><dd>Kongolo</dd>\n<dt>Country</dt><dd>CD</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 75.9 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>VS Villa</dd>\n<dt>City</dt><dd>Kalmu

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [25]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
1,-21.2078,-159.7750
2,3.7833,72.9667
7,19.7297,-155.0900
10,-21.2000,-175.2000
12,-5.3833,27.0000
23,7.4167,81.8167
27,-2.5744,150.7967
28,9.4942,50.8122
31,37.4636,-122.4286
41,-16.4833,-151.7500


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [27]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))